# My notebook

In [20]:
import pandas as pd
import json
from tqdm.notebook import tqdm
tqdm.pandas()

In [21]:
with open("msample.json", "r") as f:
  items = [json.loads(line) for line in f]
  df = pd.DataFrame({
      "id": [x["_source"]["id"] for x in items],
      "name": [x["_source"]["name"] for x in items],
      "entype": [x["_source"]["entype"] for x in items],
  })

#print(df.head())

In [22]:
#df["entype"].value_counts()

In [23]:
!pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You should consider upgrading via the '/Users/nicolo/.pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [6]:
df['Embeddings'] = df['name'].progress_apply(lambda x: model.encode(x))

  0%|          | 0/13545 [00:00<?, ?it/s]

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
def get_top5_embedding_rows(user_input):

    # convert user_input to embeddings

    vect2 = model.encode(user_input).reshape(1,-1)

    # make a copy of original dataframe
    df_temp = df.copy()




    df_temp['similarity'] = df_temp['Embeddings'].progress_apply(lambda vect1:cosine_similarity(vect1.reshape(1,-1), vect2)[0][0])


    top_5 = df_temp.sort_values('similarity',ascending=False).head(10)



    return top_5

In [24]:
#get_top5_embedding_rows('I want to watch free movies that are funny')

# Desireble output with keras model

In [ ]:

        
class MEModel(Model):
    def __init__(self):
        super().__init__()
        
        # Define layers
        entity_input = Input(shape=(1,), name='entity_string', dtype=tf.string)
        candidate_input = Input(shape=(None,), name='candidate_string', dtype=tf.string)
        embedding_layer = UniversalEmbeddingLayer()
        dot_layer = DotProductLayer()
        
        # Construct model
        entity_embedding = embedding_layer(entity_input)
        candidate_embeddings = embedding_layer(candidate_input)
        score = tf.squeeze(dot_layer([entity_embedding, candidate_embeddings]))
    
        self.model = Model(inputs=[entity_input, candidate_input], outputs=score)
        
        # Define TensorFlow functions
        @tf.function(input_signature=[[tf.TensorSpec(shape=(None, 1), dtype=tf.string),
                                       tf.TensorSpec(shape=(None, None), dtype=tf.string)]])
        def resolve(input_tensors: List[tf.Tensor]) -> List[tf.Tensor]:
            return self.model(input_tensors)
        
        self.resolve = resolve
    
    def call(self, inputs, training=None, mask=None):
        return self.resolve(inputs)
    
me_model = MEModel()

In [ ]:
tf.saved_model.save(me_model, SAVE_PATH, signature={"resolve": me_model.resolve})